In [0]:
# from google.colab import drive
# drive.mount('/content/gdrive')
# Created by Noel Dsouza dsouzinator@github

In [0]:

import urllib.request

print('Beginning file download with urllib2...')

url = "https://storage.googleapis.com/kaggle-datasets/23079/29550/asl-alphabet.zip?GoogleAccessId=web-data@kaggle-161607.iam.gserviceaccount.com&Expires=1561170803&Signature=O%2BmI5VErW7RhaFPnzF1fy2ZKfd7CP%2FSky8P0jDf%2F22%2FFzAUPeX55LGXfd%2Bb0khl%2FVdRzhaVFV%2BKi8GgmSb3EB8emYW1qho3cEcwiJswoIc2SZmf4kmVyh1nfppcYmU1EzzGdZx7Hb2YQYUT2kK4TaPm8JPnEYfV5Xp%2BXAseUQpHdf2SsqXjTQOKNPUM4%2BNLhJvNUvd1pWe2K1nq04kCJL%2BSQe%2B5KYJ6F0%2FsZW8GVA9p8FbMydp%2FgyXezJS7GaIiXdsrOpC2ExAsEx8luFoGvC7%2B21S9EH4lQzCdFXyf4sgHizUAIqgyFjnppHUnJoA8dCUnHD%2Fohnp1BrEcqEq6L1g%3D%3D"
urllib.request.urlretrieve(url, './train.zip')

Beginning file download with urllib2...


('./train.zip', <http.client.HTTPMessage at 0x7f12d873d278>)

In [0]:
import zipfile
zip_ref = zipfile.ZipFile('/content/train.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()
zip_ref = zipfile.ZipFile('/content/asl_alphabet_train.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()
zip_ref = zipfile.ZipFile('/content/asl_alphabet_test.zip', 'r')
zip_ref.extractall('./')
zip_ref.close()

In [0]:
import os
os.listdir()

['.config',
 'asl_alphabet_train',
 '.ipynb_checkpoints',
 'asl_alphabet_test.zip',
 'asl_alphabet_train.zip',
 'train.zip',
 'sample_data']

In [0]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras import regularizers
from tensorflow.keras.layers import MaxPooling2D, Dropout
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.layers import Input, Dense, Activation, BatchNormalization, Flatten, Conv2D
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.callbacks import CSVLogger, EarlyStopping
from tensorflow.keras.layers import LeakyReLU
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import os
from keras.regularizers import l2
from PIL import Image
import tensorflow.keras.backend as K
# K.set_image_dim_ordering('tf')


In [0]:
def signs_model(input_shape, output_size, alpha=0.05, reg=0.001):
  inputs = Input(shape=input_shape)

  X = Conv2D(32, kernel_size = (3, 3), strides=(1, 1),kernel_regularizer=l2(reg),
             activation=None, padding='same')(inputs)
  X = BatchNormalization()(X)
  X = LeakyReLU(alpha = alpha)(X)

  X = Conv2D(64, kernel_size = (5, 5), strides=(1, 1), kernel_regularizer=l2(reg),
             activation=None, padding='same')(X)
  X = MaxPooling2D((2, 2))(X)
  X = Dropout(0.25)(X)
  X = BatchNormalization()(X)
  X = LeakyReLU(alpha=alpha)(X)

  X = Conv2D(256, (7, 7), strides=(2, 2), kernel_regularizer=l2(reg),
             activation=None, padding='valid')(X)
  X = BatchNormalization()(X)
  X = LeakyReLU(alpha = alpha)(X)
  X = MaxPooling2D((2, 2))(X)

  X = Conv2D(256, (7, 7), strides=(2, 2), kernel_regularizer=l2(reg),
             activation=None, padding='valid')(X)
  X = BatchNormalization()(X)
  X = LeakyReLU(alpha = alpha)(X)
  
  X = Conv2D(256, (5, 5), strides=(2, 2), kernel_regularizer=l2(reg),
             activation=None, padding='valid')(X)
  X = BatchNormalization()(X)
  X = LeakyReLU(alpha = alpha)(X)
  X = MaxPooling2D((2, 2))(X)
  
#   X = Dropout(0.3)(X)

  X = Flatten()(X)
  X = Dense(250, activation='relu', kernel_regularizer=l2(reg))(X)
#   X = LeakyReLU(alpha=0.1)(X)
#   X = Dropout(0.25)(X)
  X = Dense(500, activation='relu', kernel_regularizer=l2(reg))(X)
#   X = LeakyReLU(alpha=0.1)(X)
#   X = Dropout(0.25)(X)
  X = Dense(250, activation='relu', kernel_regularizer=l2(reg))(X)
#   X = LeakyReLU(alpha=0.1)(X)
  X = Dense(output_size, activation='softmax', kernel_regularizer=l2(reg))(X)
  
  model = Model(inputs = inputs, outputs = X)


  return model

In [0]:
# resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
# tf.contrib.distribute.initialize_tpu_system(resolver)
# strategy = tf.contrib.distribute.TPUStrategy(resolver)

# with strategy.scope():
model = signs_model(input_shape=(200,200,3), output_size=29)
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])

In [0]:
# try:
#  device_name = os.environ['COLAB_TPU_ADDR']
#  TPU_ADDRESS = 'grpc://' + device_name
#  print('Found TPU at: {}'.format(TPU_ADDRESS))
# except KeyError:
#  print('TPU not found')

# tpu_model = tf.contrib.tpu.keras_to_tpu_model(
#     model,
#     strategy=tf.contrib.tpu.TPUDistributionStrategy(
#         tf.contrib.cluster_resolver.TPUClusterResolver(TPU_ADDRESS)))

In [0]:
EPOCHS = 10
BS = 32
aug = ImageDataGenerator(rotation_range=4, zoom_range=0.2, rescale=1./255,
                         width_shift_range=0.3, height_shift_range=0.3,
                         shear_range=0.15,
                         horizontal_flip=False, fill_mode="nearest")


directory = "/content/asl_alphabet_train"

image_generator = aug.flow_from_directory(directory, target_size=(200, 200), 
                    class_mode='categorical',batch_size=16)

H3 = model.fit_generator(image_generator,steps_per_epoch=2718, epochs=EPOCHS)
#                         callbacks = [EarlyStopping(monitor='train_acc', 
#                                                    mode='max', patience = 6)])


NameError: ignored

In [0]:
model.save("./big_sign2.h5")

In [0]:
H.history

{'acc': [0.19547921,
  0.4327967,
  0.5649724,
  0.6304498,
  0.66448945,
  0.6902364,
  0.70693064,
  0.7199172,
  0.73056936,
  0.7352116],
 'loss': [3.3059976734248395,
  2.4311026159951528,
  1.9682877193378985,
  1.7270495993324375,
  1.6097536885420336,
  1.5183835522360096,
  1.462926261428527,
  1.4296358941121177,
  1.3800551341771203,
  1.3682413809871936]}

In [0]:
!pip install autocorrect
from autocorrect import spell

     |████████████████████████████████| 3.6MB 2.9MB/s 
  Stored in directory: /root/.cache/pip/wheels/bf/b8/ae/704d5643f1d0637c5b87d9feccf2ee923c492b703bb0bfbb19
Successfully built autocorrect


In [0]:
spell("ne")

'ne'

In [0]:
!pip install pyspellchecker


     |████████████████████████████████| 1.9MB 2.7MB/s 


In [0]:
from spellchecker import SpellChecker

spell = SpellChecker()

# find those words that may be misspelled
misspelled = spell.unknown('hel')

for word in misspelled:
    # Get the one `most likely` answer
    print(spell.correction(word))

    # Get a list of `likely` options
#     print(spell.candidates(word))